In [6]:
from __future__ import annotations
from pathlib import Path

import numpy as np
from ase.io import read
from qtpyt.basis import Basis
from qtpyt.surface.principallayer import PrincipalSelfEnergy
from qtpyt.surface.tools import prepare_leads_matrices
from qtpyt.tools import expand_coupling, remove_pbc, rotate_couplings

from gpaw import restart
from gpaw.lcao.pwf2 import LCAOwrap
from qtpyt.lo.tools import rotate_matrix, subdiagonalize_atoms, lowdin_rotation, get_subspace

### Initialization

In [7]:
pl_path = Path("../dft/leads/")
cc_path = Path("../dft/device/")

data_folder = "../output/lowdin"
index_active_region = np.load(f"{data_folder}/index_active_region.npy")
self_energy = np.load(f"{data_folder}/self_energy.npy", allow_pickle=True)

H_leads_lcao, S_leads_lcao = np.load(pl_path / "hs_pl_k.npy")

basis_dict = {"Au": 9, "H": 5, "C": 13, "N": 13}

leads_atom = read(pl_path / "leads.xyz")
leads_basis = Basis.from_dictionary(leads_atom, basis_dict)

device_atoms = read(cc_path / "scatt.xyz")
device_basis = Basis.from_dictionary(device_atoms, basis_dict)

Nr = (1, 5, 3)
unit_cell_rep_in_leads = (5, 5, 3)

### Hamiltonian transformation from Lowdin LO to AO basis
 

In [8]:
def get_species_indices(atoms, species):
    indices = []
    for element in species:
        element_indices = atoms.symbols.search(element)
        indices.extend(element_indices)
    return sorted(indices)

lowdin = True

GPWDEVICEDIR = "../dft/device/"
GPWLEADSDIR = "../dft/leads/"
SUBDIAG_SPECIES = ("C", "N", "H")
active = {"C": [3], "N": [3]}

cc_path = Path(GPWDEVICEDIR)
pl_path = Path(GPWLEADSDIR)
gpwfile = f"{cc_path}/scatt.gpw"

atoms, calc = restart(gpwfile, txt=None)
fermi = calc.get_fermi_level()
nao_a = np.array([setup.nao for setup in calc.wfs.setups])
basis = Basis(atoms, nao_a)

lcao = LCAOwrap(calc)
H_lcao = lcao.get_hamiltonian()
S_lcao = lcao.get_overlap()
H_lcao -= fermi * S_lcao

subdiag_indices = get_species_indices(atoms, SUBDIAG_SPECIES)

Usub, eig = subdiagonalize_atoms(basis, H_lcao, S_lcao, a=subdiag_indices)
H_subdiagonalized = rotate_matrix(H_lcao, Usub)
S_subdiagonalized = rotate_matrix(S_lcao, Usub)

if lowdin:
    Ulow = lowdin_rotation(H_subdiagonalized, S_subdiagonalized, index_active_region)

    H_subdiagonalized = rotate_matrix(H_subdiagonalized, Ulow)
    S_subdiagonalized = rotate_matrix(S_subdiagonalized, Ulow)

    H_subdiagonalized = H_subdiagonalized[None, ...]
    S_subdiagonalized = S_subdiagonalized[None, ...]


else:
    H_subdiagonalized = H_subdiagonalized[None, ...]
    S_subdiagonalized = S_subdiagonalized[None, ...]


Condition number: 1.0e+04


In [13]:
def dagger(A):
    return A.conj().T

def get_inverse(U: np.ndarray) -> np.ndarray:

    U_inv = np.linalg.inv(U)
    return U_inv


def compose_forward_and_reverse(
    Usub: np.ndarray,
    Usub_inv: np.ndarray,
    Ulow: np.ndarray,
    Ulow_inv: np.ndarray,
) -> tuple[np.ndarray, np.ndarray]:
    r"""Compose total forward and reverse transforms.

    Returns
    -------
    Utot, Urev
        Forward Utot = Usub Ulow (LCAO -> Lowdin-LO),
        Reverse Urev = Utot^{-1}  (Lowdin-LO -> LCAO).

    Notes
    -----
    We compute Urev as:

    .. math::
        Urev = Ulow^{-1} Usub^{-1}

    using explicit solves rather than assuming unitarity.
    """
    Utot = Usub @ Ulow

    Urev = Ulow_inv @ Usub_inv
    return Utot, Urev


def check_inverse_pair(
    U: np.ndarray,
    Uinv: np.ndarray,
    *,
    name: str,
    atol: float = 1e-8,
) -> None:
    r"""Check that Uinv is the inverse of U on both sides.

    Notes
    -----
    We check both:
    .. math::
        \|Uinv\,U - I\| \quad \text{and} \quad \|U\,Uinv - I\|
    because for non-unitary matrices numerical errors can be asymmetric.
    """
    n = U.shape[0]
    I = np.eye(n, dtype=U.dtype)

    left = np.linalg.norm(Uinv @ U - I) / np.linalg.norm(I)
    right = np.linalg.norm(U @ Uinv - I) / np.linalg.norm(I)
    if left >= atol or right >= atol:
        raise AssertionError(
            f"{name}: inverse check failed. "
            f"relerr_left=||Uinv U - I||={left:g}, "
            f"relerr_right=||U Uinv - I||={right:g}"
        )
    else:
        print(
            f"{name}: inverse check passed. "
            f"relerr_left=||Uinv U - I||={left:g}, "
            f"relerr_right=||U Uinv - I||={right:g}"
        )

def sanity_checks(
    Usub: np.ndarray,
    Usub_inv: np.ndarray,
    Ulow: np.ndarray,
    Ulow_inv: np.ndarray,
    Urev: np.ndarray,
    S_lcao: np.ndarray,
    index_active_region: np.ndarray,
    atol: float = 1e-8,
) -> None:
    r"""Run basic numerical checks and raise AssertionError if something is off.

    Checks performed
    ----------------
    1) Usub inverse consistency
       .. math:: Usub^{-1} Usub \approx I,\;\; Usub Usub^{-1} \approx I
    2) Ulow inverse consistency (full-space embedded matrix)
       .. math:: Ulow^{-1} Ulow \approx I,\;\; Ulow Ulow^{-1} \approx I
    3) Total inverse consistency
       .. math:: Urev (Usub Ulow) \approx I
    4) Active-space overlap becomes identity after Lowdin
       .. math:: (Ulow^\dagger Usub^\dagger) S (Usub Ulow) \approx I \text{ on active block}
    """
    n = S_lcao.shape[0]
    I = np.eye(n, dtype=S_lcao.dtype)

    check_inverse_pair(Usub, Usub_inv, name="Usub", atol=atol)

    check_inverse_pair(Ulow, Ulow_inv, name="Ulow", atol=atol)

    Utot = Usub @ Ulow
    err_tot = np.linalg.norm(Urev @ Utot - I) / np.linalg.norm(I)
    if err_tot >= atol:
        raise AssertionError(f"Urev*Utot not identity: relerr={err_tot:g}")
    else:
        print(f"Urev*Utot identity check passed: relerr={err_tot:g}")

    err_tot_r = np.linalg.norm(Utot @ Urev - I) / np.linalg.norm(I)
    if err_tot_r >= atol:
        raise AssertionError(f"Utot*Urev not identity: relerr={err_tot_r:g}")
    else:
        print(f"Utot*Urev identity check passed: relerr={err_tot_r:g}")

    S_lo = rotate_matrix(S_lcao, Usub)
    S_low = rotate_matrix(S_lo, Ulow)
    idx = np.asarray(index_active_region, dtype=int)
    S_active = S_low[np.ix_(idx, idx)]
    err2 = np.linalg.norm(S_active - np.eye(len(idx), dtype=S_active.dtype))
    if err2 >= 50 * atol:
        raise AssertionError(f"Active overlap not identity after Lowdin: ||Δ||={err2:g}")
    else:
        print(f"Active overlap identity check passed after Lowdin: ||Δ||={err2:g}")

In [14]:
Usub_inv = get_inverse(Usub)
Ulow_inv = get_inverse(Ulow)

Utot, Urev = compose_forward_and_reverse(Usub, Usub_inv, Ulow, Ulow_inv)

sanity_checks(Usub, Usub_inv, Ulow, Ulow_inv, Urev, S_lcao, index_active_region)

Usub: inverse check passed. relerr_left=||Uinv U - I||=1.0757e-16, relerr_right=||U Uinv - I||=5.67974e-17
Ulow: inverse check passed. relerr_left=||Uinv U - I||=9.70902e-18, relerr_right=||U Uinv - I||=8.58548e-18
Urev*Utot identity check passed: relerr=1.08696e-16
Utot*Urev identity check passed: relerr=6.35717e-17
Active overlap identity check passed after Lowdin: ||Δ||=3.83632e-15


In [ ]:
def compute_relative_frobenius_error(A: np.ndarray, B: np.ndarray) -> float:
    r"""Relative Frobenius error: ||A-B||_F / ||A||_F."""
    num = np.linalg.norm(A - B)
    den = np.linalg.norm(A)
    return float(num / den) if den != 0 else float(num)

def hermiticity_error(A: np.ndarray) -> float:
    r"""Absolute Frobenius norm of anti-Hermitian part: ||A - A^\dagger||_F."""
    return float(np.linalg.norm(A - dagger(A)))

def hamiltonian_roundtrip_checks(
    H_lcao: np.ndarray,
    Usub: np.ndarray,
    Usub_inv: np.ndarray,
    Ulow: np.ndarray,
    Ulow_inv: np.ndarray,
    *,
    atol_rel: float = 1e-10,
    atol_herm: float = 1e-10,
) -> None:
    r"""Verify that forward+reverse basis transforms recover the original Hamiltonian.

    Checks
    ------
    1) LCAO -> LO:
       .. math:: H_{lo} = U_{sub}^\dagger H_{lcao} U_{sub}
    2) LO -> Lowdin-LO:
       .. math:: H_{low} = U_{low}^\dagger H_{lo} U_{low}
    3) Reverse back:
       .. math::
           H_{lo}^{(rt)}   = (U_{low}^{-1})^\dagger H_{low} (U_{low}^{-1})
           H_{lcao}^{(rt)} = (U_{sub}^{-1})^\dagger H_{lo}^{(rt)} (U_{sub}^{-1})

    We then compare :math:`H_{lcao}^{(rt)}` to :math:`H_{lcao}`.

    Notes
    -----
    - Use relative Frobenius error for meaningful scale-invariant comparison.
    - Also check Hermiticity drift, since numerical transforms can introduce tiny
      anti-Hermitian parts if inputs are not perfectly Hermitian.
    """
    H_lo = rotate_matrix(H_lcao, Usub)
    H_low = rotate_matrix(H_lo, Ulow)

    H_lo_rt = rotate_matrix(H_low, Ulow_inv)
    H_lcao_rt = rotate_matrix(H_lo_rt, Usub_inv)

    err_rel = compute_relative_frobenius_error(H_lcao, H_lcao_rt)
    if err_rel >= atol_rel:
        raise AssertionError(
            f"H round-trip failed: rel Fro error ||H - H_rt||/||H|| = {err_rel:g}"
        )
    else:
        print(
            f"H round-trip check passed: rel Fro error ||H - H_rt||/||H|| = {err_rel:g}"
        )

    herm_in = hermiticity_error(H_lcao)
    herm_rt = hermiticity_error(H_lcao_rt)
    if herm_rt >= max(atol_herm, 10 * herm_in):
        raise AssertionError(
            f"H Hermiticity drift too large: ||H_rt - H_rt^†|| = {herm_rt:g} "
            f"(input anti-Herm norm was {herm_in:g})"
        )
    else:
        print(
            f"H Hermiticity drift check passed: ||H_rt - H_rt^†|| = {herm_rt:g} "
            f"(input anti-Herm norm was {herm_in:g})"
        )

In [18]:
hamiltonian_roundtrip_checks(
    H_lcao,
    Usub, Usub_inv,
    Ulow, Ulow_inv,
    atol_rel=1e-10,
    atol_herm=1e-10,
)
hamiltonian_roundtrip_checks(
    S_lcao,
    Usub, Usub_inv,
    Ulow, Ulow_inv,
    atol_rel=1e-10,
    atol_herm=1e-10,
)

H round-trip check passed: rel Fro error ||H - H_rt||/||H|| = 1.96184e-16
H Hermiticity drift check passed: ||H_rt - H_rt^†|| = 7.75323e-14 (input anti-Herm norm was 0)
H round-trip check passed: rel Fro error ||H - H_rt||/||H|| = 1.39164e-16
H Hermiticity drift check passed: ||H_rt - H_rt^†|| = 5.23479e-15 (input anti-Herm norm was 0)


### Self-energy Transformation from Lowdin LO to AO basis

In [ ]:
kpts_t, h_leads_kii, s_leads_kii, h_leads_kij, s_leads_kij = prepare_leads_matrices(
    H_leads_lcao,
    S_leads_lcao,
    unit_cell_rep_in_leads,
    align=(0, H_subdiagonalized[0, 0, 0]),
)

remove_pbc(device_basis, H_subdiagonalized)
remove_pbc(device_basis, S_subdiagonalized)
# Initialize self-energy list for left and right leads
self_energy = [None, None, None]
self_energy[0] = PrincipalSelfEnergy(
    kpts_t, (h_leads_kii, s_leads_kii), (h_leads_kij, s_leads_kij), Nr=Nr
)
self_energy[1] = PrincipalSelfEnergy(
    kpts_t, (h_leads_kii, s_leads_kii), (h_leads_kij, s_leads_kij), Nr=Nr, id="right"
)

# Rotate the couplings for the leads based on the specified basis and repetition Nr
rotate_couplings(leads_basis, self_energy[0], Nr)
rotate_couplings(leads_basis, self_energy[1], Nr)

# expand to dimension of scattering
expand_coupling(self_energy[0], len(H_subdiagonalized[0]))
expand_coupling(self_energy[1], len(H_subdiagonalized[0]), id="right")
